In this file I will be building and evaluating my model, based on the data made with the other jupyter notebooks.

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\bramm\AppData\Local\Temp\ipykernel_60976\2312963102.py:3: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('NS_Data\combined_trein_data_feature_engineered.csv')


,Stop:Departure delay,departure year,departure month,departure day,departure hour,departure minute,departure weekday,season,Service:Type_Alpen Express,Service:Type_Bus,...,Service:Company_Blauwnet,Service:Company_Breng,Service:Company_DB,Service:Company_Eurobahn,Service:Company_NMBS,Service:Company_NS,Service:Company_R-net,Service:Company_Railexpert,Service:Company_Valleilijn,station_encoded
0,1.0,2019.0,1.0,1.0,2.0,0.0,1.0,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,287
1,0.0,2019.0,1.0,1.0,2.0,12.0,1.0,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,89
2,1.0,2019.0,1.0,1.0,2.0,21.0,1.0,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,95
3,0.0,2019.0,1.0,1.0,2.0,45.0,1.0,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,225
4,0.0,2019.0,1.0,1.0,3.0,2.0,1.0,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,302


Target:
Stop:Departure delay — a continuous value (so it’s a regression problem).

Goal:
Predict future delay given time, station, service type, etc.

Important: Because it’s time-dependent, always split data chronologically, not randomly — you want the model to learn from past → predict future.